# Quick Test - Verify Installation
# ================================

This notebook verifies that all dependencies are installed correctly and tests the basic functionality of the Adaptive Integrated Grad-CAM implementation.

## Steps:
1. Check all dependencies
2. Test GPU availability
3. Load pretrained model
4. Test Adaptive Integrated Grad-CAM
5. Test evaluation metrics

In [ ]:
# Step 1: Check Dependencies
print("="*70)
print("ADAPTIVE INTEGRATED GRAD-CAM - INSTALLATION TEST")
print("="*70)

print("\n[1/5] Checking dependencies...")
import sys

try:
    import torch
    import torchvision.models as models
    import cv2
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
    import numpy as np
    from PIL import Image
    from tqdm import tqdm
    print("✓ All dependencies installed!")
    print(f"  - PyTorch: {torch.__version__}")
    print(f"  - NumPy: {np.__version__}")
    print(f"  - Pandas: {pd.__version__}")
except ImportError as e:
    print(f"✗ Missing dependency: {e}")
    sys.exit(1)

In [ ]:
# Step 2: Check GPU Availability
print("\n[2/5] Checking GPU availability...")

if torch.cuda.is_available():
    device = 'cuda'
    print(f"✓ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"  - CUDA version: {torch.version.cuda}")
    print(f"  - GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = 'cpu'
    print("⚠ No GPU detected, using CPU (will be slower)")

In [ ]:
# Step 3: Load Pretrained Model
print("\n[3/5] Loading pretrained model...")

try:
    model = models.resnet50(weights='IMAGENET1K_V1')
    target_layer = model.layer4[-1]
    model.eval()
    model.to(device)
    print("✓ Model loaded successfully (ResNet-50)")
    print(f"  - Parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"  - Target layer: layer4[-1]")
except Exception as e:
    print(f"✗ Error loading model: {e}")
    sys.exit(1)

In [ ]:
# Step 4: Test Adaptive Integrated Grad-CAM
print("\n[4/5] Testing Adaptive Integrated Grad-CAM...")

try:
    from adaptive_integrated_gradcam import AdaptiveIntegratedGradCAM
    
    # Create method instance
    adaptive_method = AdaptiveIntegratedGradCAM(
        model, 
        target_layer,
        min_steps=10,
        max_steps=50,
        variance_threshold=0.1,
        convergence_threshold=0.05
    )
    
    # Create random test image
    test_image = torch.randn(1, 3, 224, 224).to(device)
    
    # Generate CAM
    cam, num_steps = adaptive_method.generate_cam(test_image, use_adaptive=True)
    
    print(f"✓ Adaptive method works!")
    print(f"  - Generated CAM shape: {cam.shape}")
    print(f"  - Steps allocated: {num_steps}")
    if len(adaptive_method.step_allocation_history) > 0:
        history = adaptive_method.step_allocation_history[-1]
        print(f"  - Gradient variance: {history['gradient_variance']:.4f}")
        print(f"  - Attribution change: {history['attribution_change']:.4f}")
    
except Exception as e:
    print(f"✗ Error in adaptive method: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)

In [ ]:
# Step 5: Test Evaluation Metrics
print("\n[5/5] Testing evaluation metrics...")

try:
    from evaluation_metrics import AttributionEvaluator
    
    evaluator = AttributionEvaluator(model, device)
    
    # Quick deletion test
    _, deletion_auc = evaluator.deletion_metric(
        test_image, cam, target_class=0, steps=5
    )
    
    print(f"✓ Evaluation metrics work!")
    print(f"  - Deletion AUC: {deletion_auc:.4f}")
    
except Exception as e:
    print(f"✗ Error in evaluation: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)

In [ ]:
# Success Summary
print("\n" + "="*70)
print("SUCCESS! All tests passed ✓")
print("="*70)
print("\nYou're ready to start your experiments!")
print("\nNext steps:")
print("  1. Create dataset: Run notebook 2_create_sample_dataset.ipynb")
print("  2. Run experiments: Run notebook 3_run_batch_experiments.ipynb")
print("  3. Analyze results: Run notebook 4_analyze_results.ipynb")
print("\nGood luck with your PhD project! 🚀")
print("="*70 + "\n")